# Multi-Task Test
At first we have to install the newest version of gpCAM

In [1]:
##first install the newest version of fvgp
#!pip install gpCAM==...

## Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from gpcam import fvGPOptimizer
import plotly.graph_objects as go
from itertools import product

%load_ext autoreload
%autoreload 2

## Data

In [3]:
data = np.load("./data/sim_variable_mod.npy")
sparsification = 16

x_data3 = data[:,5:][::sparsification]
y_data3 = data[:,0:3][::sparsification]

#it is good practice to check the format of the data
print(x_data3.shape)
print(y_data3.shape)

(396, 3)
(396, 3)


In [4]:
x = np.linspace(30,100,100)
y = np.linspace(40,130,100)
x_pred3D = np.asarray(list(product(x, y)))
x_pred3D = np.column_stack([x_pred3D, np.zeros((len(x_pred3D),1)) + 300.])

## Plotting

In [5]:
def scatter(x,y,z,size=3, color = 1):
    #if not color: color = z
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z,mode='markers',marker=dict(color=color, size = size)))
    
    
    fig.update_layout(autosize=False,
                  width=800, height=800,
                  font=dict(size=18,),
                  margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [ ]:
scatter(x_data3[:,0],x_data3[:,1],x_data3[:,2], size = 5, color = y_data3[:,0])
scatter(x_data3[:,0],x_data3[:,1],x_data3[:,2], size = 5, color = y_data3[:,1])
scatter(x_data3[:,0],x_data3[:,1],x_data3[:,2], size = 5, color = y_data3[:,2])

## A simple kernel definition
It is vital in the multi-task case to think hard about kernel design. The kernel is now a function
over $\mathcal{X} \times \mathcal{X} \times T \times T$, where $\mathcal{X}$ is the input and $T$ is the output space. Print the input into kernel, it will have the dimensionality of this cartesian product space. 
The default kernel in fvgp is a deep kernel, which can be good, but there is no guarantee.
To use the default kernel, pytorch has to be installed manually (pip install torch).

In [7]:
#As imple kernel, that won't lead to good performance because its stationary
def mkernel(x1,x2,hps,obj):
    d = obj._get_distance_matrix(x1,x2)
    return hps[0] * obj.matern_kernel_diff1(d,hps[1])

## Initialization

In [8]:
#This is where things get a littlecomplicated. What's with all those numbers there.
#This input space is 3-dimensional, the output space has 3 tasks, but is still 1-dimensional
#in the fvgp world. Therefore fvGP(3,1,3, ...), for 3 dim input, 1 dim output, with 3 outputs 

my_gp2 = fvGPOptimizer(x_data3,y_data3, 1,init_hyperparameters=np.ones((2)), info = True,
              #gp_kernel_function=mkernel #what happens if comment this out? (adjust bounds below)
             )
#change this based on kernel choice
hps_bounds = my_gp2.hps_bounds 
#hps_bounds = np.array([[0.001,10000.],[1.,1000.]])

#my_gp2.update_gp_data(x_data3,y_data3)
print("Global Training in progress")
#my_gp2.train(hps_bounds, max_iter = 2)

Global Training in progress


## Prediction

In [9]:
#first task
mean1 = my_gp2.posterior_mean(x_pred3D, x_out = np.zeros((1,1)))["f(x)"]
var1 =  my_gp2.posterior_covariance(x_pred3D, x_out = np.zeros((1,1)))["v(x)"]

#second task
mean2 = my_gp2.posterior_mean(x_pred3D, x_out = np.zeros((1,1)) + 1)["f(x)"]
var2 =  my_gp2.posterior_covariance(x_pred3D, x_out = np.zeros((1,1))+1)["v(x)"]

#third task
mean3 = my_gp2.posterior_mean(x_pred3D, x_out = np.zeros((1,1)) + 2)["f(x)"]
var3 =  my_gp2.posterior_covariance(x_pred3D, x_out = np.zeros((1,1))+2)["v(x)"]

type: Wrong dimensionality of the input points x_pred.

In [21]:
#extract data point to compare to:
index300 = np.where(x_data3[:,2]==300.)
imageX_data = x_data3[index300]
imageY_data = y_data3[index300]
print(y_data3)

[[0.43295063 0.46441413 0.        ]
 [0.43576938 0.43058881 0.        ]
 [0.40181642 0.49974507 0.        ]
 ...
 [0.5088312  0.15046951 0.2201406 ]
 [0.27935588 0.15296478 0.18567747]
 [0.51624648 0.14485927 0.25474667]]


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_pred3D[:,0],y=x_pred3D[:,1], z=mean1,
                             mode='markers',marker=dict(color=mean1, size = 5)))
fig.add_trace(go.Scatter3d(x=imageX_data[:,0], y=imageX_data[:,1] , z=imageY_data[:,0],
                           mode='markers',marker=dict(color=imageY_data[:,0], size = 5)))
fig.update_layout(autosize=False,
                  width=800, height=800,
                  font=dict(size=18,),
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()



fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_pred3D[:,0], y=x_pred3D[:,1], z=mean2,
                           mode='markers',marker=dict(color=mean2, size = 5)))
fig.add_trace(go.Scatter3d(x=imageX_data[:,0], y=imageX_data[:,1], z=imageY_data[:,1],
                           mode='markers',marker=dict(color=imageY_data[:,1], size = 5)))
fig.update_layout(autosize=False,
                  width=800, height=800,
                  font=dict(size=18,),
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_pred3D[:,0], y=x_pred3D[:,1], z=mean3,
                           mode='markers',marker=dict(color=mean3, size = 5)))
fig.add_trace(go.Scatter3d(x=imageX_data[:,0], y=imageX_data[:,1], z=imageY_data[:,2],
                           mode='markers',marker=dict(color=imageY_data[:,2], size = 5)))
fig.update_layout(autosize=False,
                  width=800, height=800,
                  font=dict(size=18,),
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

